Base de modulos y funciones.

In [1]:
import pandas as pd
import re
import string
import numpy as np


def standardise_column_names(df, remove_punct=True):
    """ Converts all DataFrame column names to lower case replacing
    whitespace of any length with a single underscore. Can also strip
    all punctuation from column names.

    Parameters
    ----------
    df: pandas.DataFrame
        DataFrame with non-standardised column names.
    remove_punct: bool (default True)
        If True will remove all punctuation from column names.

    Returns
    -------
    df: pandas.DataFrame
        DataFrame with standardised column names.
    Example
    -------
    >>> df = pd.DataFrame({'Column With Spaces': [1,2,3,4,5],
                           'Column-With-Hyphens&Others/': [6,7,8,9,10],
                           'Too    Many Spaces': [11,12,13,14,15],
                           })
    >>> df = standardise_column_names(df)
    >>> print(df.columns)
    Index(['column_with_spaces',
           'column_with_hyphens_others',
           'too_many_spaces'], dtype='object')
    """

    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))

    for c in df.columns:
        c_mod = c.lower()
        if remove_punct:
            c_mod = c_mod.translate(translator)
        c_mod = '_'.join(c_mod.split(' '))
        if c_mod[-1] == '_':
            c_mod = c_mod[:-1]
        c_mod = re.sub(r'\_+', '_', c_mod)
        df.rename({c: c_mod}, inplace=True, axis=1)
    return df

In [2]:
# Leemos el archivo
df = pd.read_csv('crucero_1.csv', header=1)

In [3]:
# Eliminamos filas y columnas vacias
df = df.dropna(axis=0, how='all')
df = df.dropna(axis=1, how='all')


In [4]:
# Cambiamos formato fecha
df['yyyy-mm-ddThh:mm:ss.sss'] = df['yyyy-mm-ddThh:mm:ss.sss'].apply(pd.to_datetime)


In [5]:
# Rellenamos vacios con datos de filas superiores
df = df.fillna(method='ffill')

In [6]:
# Arreglamos los nombres de las columnas
df = standardise_column_names(df)
print(df)

             cruise  station type yyyy_mm_ddthh_mm_ss_sss  \
0      Crucero 2016     37.0    C     2016-10-25 12:10:00   
1      Crucero 2016     37.0    C     2016-10-25 12:10:00   
2      Crucero 2016     37.0    C     2016-10-25 12:10:00   
3      Crucero 2016     37.0    C     2016-10-25 12:10:00   
4      Crucero 2016     37.0    C     2016-10-25 12:10:00   
...             ...      ...  ...                     ...   
25098  Crucero 2018     25.0    C     2018-07-23 21:53:00   
25099  Crucero 2018     25.0    C     2018-07-23 21:53:00   
25100  Crucero 2018     25.0    C     2018-07-23 21:53:00   
25101  Crucero 2018     25.0    C     2018-07-23 21:53:00   
25102  Crucero 2018     25.0    C     2018-07-23 21:53:00   

       longitude_degrees_east  latitude_degrees_north  bot_depth_m ship  \
0                    73.05760                46.17430        270.0   A1   
1                    73.05760                46.17430        270.0   A1   
2                    73.05760             

In [7]:
# Agrupamos por las variables del viaje en id y creamos df2 sin variables de las mediciones.
df["id"] = df.groupby(df[['cruise', 'ship', 'station', 'yyyy_mm_ddthh_mm_ss_sss',
                          'longitude_degrees_east', 'latitude_degrees_north', 'bot_depth_m']]
                      .apply(frozenset, axis=1)).ngroup()

df2 = df.drop(['depth_m', 'fluorescence_wet_labs_eco_afl_fl_mg_m_3',
               'dissolved_oxygen_ml_l', 'temperature_deg_c', 'salinity_practical_psu'],
              axis=1)
print(df2)

             cruise  station type yyyy_mm_ddthh_mm_ss_sss  \
0      Crucero 2016     37.0    C     2016-10-25 12:10:00   
1      Crucero 2016     37.0    C     2016-10-25 12:10:00   
2      Crucero 2016     37.0    C     2016-10-25 12:10:00   
3      Crucero 2016     37.0    C     2016-10-25 12:10:00   
4      Crucero 2016     37.0    C     2016-10-25 12:10:00   
...             ...      ...  ...                     ...   
25098  Crucero 2018     25.0    C     2018-07-23 21:53:00   
25099  Crucero 2018     25.0    C     2018-07-23 21:53:00   
25100  Crucero 2018     25.0    C     2018-07-23 21:53:00   
25101  Crucero 2018     25.0    C     2018-07-23 21:53:00   
25102  Crucero 2018     25.0    C     2018-07-23 21:53:00   

       longitude_degrees_east  latitude_degrees_north  bot_depth_m ship  id  
0                    73.05760                46.17430        270.0   A1   0  
1                    73.05760                46.17430        270.0   A1   0  
2                    73.05760    

In [8]:
# Creamos df3 con variables de las mediciones y con id que linkea tablas en formato 1 a muchos
# donde 1 es en df2
df3 = df.drop(['cruise', 'ship', 'station', 'type', 'yyyy_mm_ddthh_mm_ss_sss',
              'longitude_degrees_east', 'latitude_degrees_north', 'bot_depth_m'], axis=1)
print(df3)

       depth_m  fluorescence_wet_labs_eco_afl_fl_mg_m_3  \
0        0.497                                      0.0   
1        1.010                                      0.0   
2        1.516                                      0.0   
3        2.012                                      0.0   
4        2.523                                      0.0   
...        ...                                      ...   
25098  298.000                                      0.0   
25099  298.500                                      0.0   
25100  299.000                                      0.0   
25101  299.500                                      0.0   
25102  300.000                                      0.0   

       dissolved_oxygen_ml_l  temperature_deg_c  salinity_practical_psu  id  
0                    6.88523             7.3546                 30.7609   0  
1                    6.90105             7.2775                 30.6860   0  
2                    6.90176             7.2760          

In [9]:
# Stackeamos las variables con llaves id y depth
df4 = df3.set_index(['id', 'depth_m'])
df4 = df4.stack().rename_axis(['id', 'depth_m', 'variable']).rename('value').reset_index()
print(df4)

        id  depth_m                                 variable     value
0        0    0.497  fluorescence_wet_labs_eco_afl_fl_mg_m_3   0.00000
1        0    0.497                    dissolved_oxygen_ml_l   6.88523
2        0    0.497                        temperature_deg_c   7.35460
3        0    0.497                   salinity_practical_psu  30.76090
4        0    1.010  fluorescence_wet_labs_eco_afl_fl_mg_m_3   0.00000
...     ..      ...                                      ...       ...
100395  58  299.500                   salinity_practical_psu  33.21230
100396  58  300.000  fluorescence_wet_labs_eco_afl_fl_mg_m_3   0.00000
100397  58  300.000                    dissolved_oxygen_ml_l   6.71302
100398  58  300.000                        temperature_deg_c   7.76370
100399  58  300.000                   salinity_practical_psu  33.21310

[100400 rows x 4 columns]


In [10]:
# Obtenemos los datos de cruseros por año
print(df['cruise'].value_counts())
print('El crusero de 2018 tiene mas datos')

Crucero 2018    9830
Crucero 2016    7774
Crucero 2017    7496
Name: cruise, dtype: int64
El crusero de 2018 tiene mas datos


In [14]:
# Obtenemos los datos de las horas agrupandolas
f = df
f['time'] = df['yyyy_mm_ddthh_mm_ss_sss'].dt.strftime('%H')
print(f['time'].value_counts())
print(' Las 22 horas poseen la mayor cantidad de datos')

22    2789
14    2199
20    2156
12    1991
21    1691
17    1505
02    1325
16    1277
23    1248
08    1218
03    1156
09    1099
06     967
04     731
13     690
11     537
19     534
10     404
07     400
01     394
00     346
05     226
15     217
Name: time, dtype: int64
 La hora con mayor cantidad de datos es 14:51:00 


In [15]:
# Obtenemos variables segun rango de profundidad.
print('El promedio de las variables por profundidad es: ')
print(df3.groupby(pd.cut(df3["depth_m"], np.arange(0, 500, 10))).mean().drop('id', axis=1))

El promedio de las variables por profundidad es: 
               depth_m  fluorescence_wet_labs_eco_afl_fl_mg_m_3  \
depth_m                                                           
(0, 10]       5.188698                                 0.000876   
(10, 20]     15.136645                                 0.005172   
(20, 30]     25.165145                                 0.000000   
(30, 40]     35.193853                                 0.000000   
(40, 50]     45.222678                                 0.000000   
(50, 60]     55.155023                                 0.000000   
(60, 70]     65.122051                                 0.000000   
(70, 80]     75.151827                                 0.000000   
(80, 90]     85.181785                                 0.000000   
(90, 100]    95.211847                                 0.000000   
(100, 110]  105.118456                                 0.000000   
(110, 120]  115.039443                                 0.000000   
(120, 130]  

In [16]:
# Obtenemos variables segun año a traves de agrupar por crusero.
X = df.groupby(df['cruise']).mean()
print('El promedio de las variables por años es: ')
print(X[['fluorescence_wet_labs_eco_afl_fl_mg_m_3', 'dissolved_oxygen_ml_l',
         'temperature_deg_c', 'salinity_practical_psu']])

El promedio de las variables por años es: 
              fluorescence_wet_labs_eco_afl_fl_mg_m_3  dissolved_oxygen_ml_l  \
cruise                                                                         
Crucero 2016                                 0.000129               6.803122   
Crucero 2017                                 0.000000               6.782033   
Crucero 2018                                 0.000610               6.822331   

              temperature_deg_c  salinity_practical_psu  
cruise                                                   
Crucero 2016           7.425740               32.385250  
Crucero 2017           7.651004               32.161600  
Crucero 2018           7.362178               32.247831  
